In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
import kagglehub
from kagglehub import KaggleDatasetAdapter
import pandas as pd
import numpy as np
import re
import tqdm
import logging
from data.make_data import get_links, is_article, save_links

np.random.seed(41)
logger = logging.getLogger(__name__)


/workspaces/hot-links/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# Download latest version
path = kagglehub.dataset_download("bhavikjikadara/bbc-news-articles")

print("Path to dataset files:", path)

Path to dataset files: /home/vscode/.cache/kagglehub/datasets/bhavikjikadara/bbc-news-articles/versions/1


: 

In [ ]:
df = kagglehub.dataset_load(
    KaggleDatasetAdapter.PANDAS,
    "adityakharosekar2/guardian-news-articles",
    'guardian_articles.csv'
)

In [ ]:
df.head()

,article_id,sectionName,webTitle,webUrl,bodyContent,webPublicationDate,id
0,us-news/2016/jan/31/iowa-caucus-underdog-candi...,US news,Iowa underdogs put on brave faces despite all ...,https://www.theguardian.com/us-news/2016/jan/3...,As polling day looms and the cameras turn only...,2016-01-31T23:53:37Z,1
1,us-news/2016/jan/31/iowa-caucus-worlds-most-pa...,US news,Iowa caucus: hologram eagle and Jesus star on ...,https://www.theguardian.com/us-news/2016/jan/3...,"In Des Moines on Sunday, the Guardian was give...",2016-01-31T23:46:28Z,2
2,world/2016/jan/31/tanzania-britsh-helicopter-p...,World news,British pilot in Tanzania 'manoeuvred ​to save...,https://www.theguardian.com/world/2016/jan/31/...,A British pilot who was shot dead by an elepha...,2016-01-31T23:43:48Z,3
3,football/2016/jan/31/late-winner-gets-usa-off-...,Football,USA 3-2 Iceland | International friendly match...,https://www.theguardian.com/football/2016/jan/...,USA took a step toward shaking off the ghosts ...,2016-01-31T23:30:49Z,4
4,football/2016/jan/31/blackburn-paul-lambert-ox...,Football,Reinvigorated Paul Lambert reflects after impr...,https://www.theguardian.com/football/2016/jan/...,"The clean-shaven, spectacle free and suspiciou...",2016-01-31T22:30:10Z,5


In [ ]:
df.to_csv('../data/raw/gurdian_articles.csv', index=False)

In [3]:
df = pd.read_csv('../data/raw/gurdian_articles.csv')

In [4]:
df.sample(3)[['webUrl', 'id']].apply(get_links,axis =1)

54801     {'url': 'https://www.theguardian.com/commentis...
114660    {'url': 'https://www.theguardian.com/world/202...
95843     {'url': 'https://www.theguardian.com/lifeandst...
dtype: object

In [54]:
n = 10

test = df.sample(n)[['webUrl', 'id']]

In [ ]:
test_out = test.apply(get_links, metadata = True, axis = 1)

In [59]:
test_out.head()

3027      ({'url': 'https://www.theguardian.com/books/20...
11224     ({'url': 'https://www.theguardian.com/film/201...
148150    ({'url': 'https://www.theguardian.com/sport/20...
57793     ({'url': 'https://www.theguardian.com/world/20...
147283    ({'url': 'https://www.theguardian.com/environm...
dtype: object

In [60]:
df.sample(10).iloc[1,:][['webUrl', 'id']]

webUrl    https://www.theguardian.com/society/2022/may/2...
id                                                   147271
Name: 147270, dtype: object

In [61]:
res = get_links(df.sample(10).iloc[1,:][['webUrl', 'id']])

for l in res['links']:
    if is_article(l['href']):
        print(l['href'], l['link'])

https://www.theguardian.com/education/academies Academies
https://www.theguardian.com/profile/richardadams Richard Adams
https://www.theguardian.com/profile/caelainn-barr Caelainn Barr
https://www.theguardian.com/teacher-network/2016/sep/01/ask-the-experts-do-multi-academy-trusts-really-improve-school-results multi-academy trusts
https://www.theguardian.com/education/2018/jan/22/rose-hill-primary-oxford-an-orphan-school-at-the-sharp-end-of-academisation ‘We feel unwanted’: an ‘orphan’ school at the sharp end of academisationRead more
https://www.theguardian.com/education/schools Schools
https://www.theguardian.com/education/2017/oct/21/collapsing-wakefield-city-academies-trust-asset-stripped-schools-millions-say-furious-parents Collapsing academy trust ‘asset-stripped its schools of millions’Read more


In [10]:
save_links(res)

## Define a seed/test set

In [5]:
df.sort_values('webPublicationDate',ascending=False,inplace=True)
# Sort to have most recent at the top

In [17]:
n_test = 100

df_test = df.iloc[:n_test]

In [18]:
test_set_links = df_test[['webUrl', 'id']].apply(get_links,axis=1,v = False)

In [19]:
test_set_links

147918    {'url': 'https://www.theguardian.com/environme...
147919    {'url': 'https://www.theguardian.com/world/202...
147920    {'url': 'https://www.theguardian.com/australia...
147921    {'url': 'https://www.theguardian.com/australia...
147922    {'url': 'https://www.theguardian.com/lifeandst...
                                ...                        
148013    {'url': 'https://www.theguardian.com/world/202...
148014    {'url': 'https://www.theguardian.com/football/...
148015    {'url': 'https://www.theguardian.com/football/...
148016    {'url': 'https://www.theguardian.com/uk-news/2...
148017    {'url': 'https://www.theguardian.com/sport/202...
Length: 100, dtype: object

In [8]:
def extract_links(blob):
    return list(filter(is_article, [d['href'] for d in blob['links']]))

In [9]:
def is_article_in_df(link):
    return link in df['webUrl'].values

In [11]:
is_article_in_df('https://www.theguardian.com/world/2024/jun/20/ukraine-war-russia-accused-of-using-banned-phosphorus-bombs-in-battle-for-bakhmut')

False

In [9]:
is_article_in_df(df.iloc[0]['webUrl'])


True

In [ ]:
def print_status(test_set_links):
    status_string = 'Looking in {:d} articles, {:d} links to articles found, of which {:d} are unique {:.2f}%\n\
    And {:d} are already in DB ({:.2f}%)\n\
            {:d} left to do'\
    .format(test_set_links.shape[0], len(flat_list), len(flat_list_unique), (100 * (len(flat_list_unique) / float(len(flat_list)))),
            n_in_db, (100 * (n_in_db / float(len(flat_list_unique)))),(len(flat_list_unique) - n_in_db))
    print(status_string)
    logging.info(status_string)


In [13]:
print_status(test_set_links)

NameError: name 'flat_list' is not defined

## Snowball Sample

In [ ]:
from tqdm.auto import tqdm
tqdm.pandas()
test_set_links = df_test[['webUrl', 'id']].progress_apply(get_links,axis=1)

df_test['links'] = test_set_links
df_test.to_csv('../data/interim/seed_set.csv')
# Save links in seed set

gen = 0

while True:

    flat_list = [item for sublist in test_set_links.apply(extract_links) for item in sublist]

    flat_list_unique = list(set(flat_list))

    is_list_in_db = list(map(is_article_in_df, flat_list_unique))

    n_in_db = sum(is_list_in_db)

    ratio = n_in_db / float(len(flat_list_unique)) 
    
    print_status(test_set_links)
    
    # test_set_links = [l for l,flag in zip(test_set_links,is_list_in_db) if flag == False]
    # Only keep links to articles not yet in the DB
    
    flat_list_unique = [l for l,flag in zip(flat_list_unique,is_list_in_db) if flag == False]
    # Check this

    test_set_links = pd.Series(map(lambda x:get_links(x,metadata=True),tqdm([{'webUrl':url, 'id' : -99999} for url in flat_list_unique])))

    # Add these parsed articles into the DB
    df = pd.concat([df,pd.DataFrame(data = {'article_id' : 'XXX', 'sectionName' : 'XXX', 'webTitle' : test_set_links.apply(lambda x:x['title']), 
                    'webUrl' : test_set_links.apply(lambda x:x['url']), 'bodyContent' : test_set_links.apply(lambda x:x['bodyContent']),
                    'webPublicationDate' : test_set_links.apply(lambda x:x['date']),'id' : 999999})])

    if ratio > 0.4 or gen > 4:
        break
    
    gen+=1

  0%|          | 0/100 [00:00<?, ?it/s]

100%|██████████| 100/100 [00:05<00:00, 16.87it/s]


Looking in 100 articles, 932 links to articles found, of which 679 are unique 72.85%
    And 159 are already in DB (23.42%)
            520 left to do


100%|██████████| 520/520 [01:16<00:00,  6.76it/s]


Looking in 520 articles, 3536 links to articles found, of which 1651 are unique 46.69%
    And 441 are already in DB (26.71%)
            1210 left to do


100%|██████████| 1210/1210 [02:12<00:00,  9.13it/s]


Looking in 1210 articles, 7472 links to articles found, of which 2712 are unique 36.30%
    And 962 are already in DB (35.47%)
            1750 left to do


100%|██████████| 1750/1750 [02:58<00:00,  9.82it/s]


Looking in 1750 articles, 16965 links to articles found, of which 3545 are unique 20.90%
    And 1487 are already in DB (41.95%)
            2058 left to do


100%|██████████| 2058/2058 [03:18<00:00, 10.38it/s]


In [13]:
df.tail()

,article_id,sectionName,webTitle,webUrl,bodyContent,webPublicationDate,id
2053,XXX,XXX,429 Too Many Requests,https://www.theguardian.com/world/libya-flood-...,,NaT,999999
2054,XXX,XXX,429 Too Many Requests,https://www.theguardian.com/global/commentisfr...,,NaT,999999
2055,XXX,XXX,Giant windfarm off Norfolk coast halted due to...,https://www.theguardian.com/business/2023/jul/...,The government’s green energy ambitions have b...,2023-07-20 13:25:00,999999
2056,XXX,XXX,429 Too Many Requests,https://www.theguardian.com/technology/2025/ju...,,NaT,999999
2057,XXX,XXX,429 Too Many Requests,https://www.theguardian.com/technology/2017/ma...,,NaT,999999


In [ ]:
df.shape

(150359, 7)

In [ ]:
df.head()

,article_id,sectionName,webTitle,webUrl,bodyContent,webPublicationDate,id
147918,environment/2022/jun/30/former-australian-chie...,Environment,Former Australian chief scientist to head revi...,https://www.theguardian.com/environment/2022/j...,The former Australian chief scientist and seni...,2022-06-30T23:53:09Z,147919
147919,world/2022/jul/01/we-were-too-lenient-on-pro-d...,World news,We were too lenient on pro-democracy politicia...,https://www.theguardian.com/world/2022/jul/01/...,A senior member of Hong Kong’s incoming admini...,2022-06-30T23:52:37Z,147920
147920,australia-news/2022/jul/01/south-australian-li...,Australia news,South Australian Liberal leader and state MPs ...,https://www.theguardian.com/australia-news/202...,A week after Roe v Wade was overturned in the ...,2022-06-30T23:33:09Z,147921
147921,australia-news/2022/jul/01/john-barilaros-deci...,Australia news,John Barilaro’s decision may stem some bleedin...,https://www.theguardian.com/australia-news/202...,John Barilaro didn’t want to continue being “a...,2022-06-30T23:23:38Z,147922
147922,lifeandstyle/2022/jul/01/breastfeeding-cogniti...,Life and style,Breastfeeding improves cognitive ability for c...,https://www.theguardian.com/lifeandstyle/2022/...,Children of poorer mothers who breastfeed are ...,2022-06-30T23:01:03Z,147923


In [16]:
df.to_csv('../data/interim/snowballed.csv')

In [15]:
df.drop_duplicates(subset=['webUrl'], inplace=False).shape

(155366, 7)

In [14]:
df.shape

(155377, 7)